In [4]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 2.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Drug_Tag

/content/drive/MyDrive/Drug_Tag


In [ ]:
!wget "https://www.dropbox.com/scl/fi/525gv6tmdi3n32mipo6mr/input.zip?rlkey=5jdsxahphk2ped5wxbxnv0n4y&dl=1" -O input.zip

!unzip input.zip

--2024-03-28 06:41:59--  https://www.dropbox.com/scl/fi/525gv6tmdi3n32mipo6mr/input.zip?rlkey=5jdsxahphk2ped5wxbxnv0n4y&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc2519e77d68a8d6f582ee5ace3.dl.dropboxusercontent.com/cd/0/inline/CP4JgMcl2hcFai-F7XtXzX49EIqHtUI0sOIX_1W6BZFz5zF-_0g2IECZYb-CWdd2ADCCp1oNkEVoe-4yxLadbfZImR4_FJ6iDl-JsF0qcfZff7d9YHtq7Ld7U9GjhNi0glShMtfn6ou3COEXa1OlkyvP/file?dl=1# [following]
--2024-03-28 06:42:00--  https://ucc2519e77d68a8d6f582ee5ace3.dl.dropboxusercontent.com/cd/0/inline/CP4JgMcl2hcFai-F7XtXzX49EIqHtUI0sOIX_1W6BZFz5zF-_0g2IECZYb-CWdd2ADCCp1oNkEVoe-4yxLadbfZImR4_FJ6iDl-JsF0qcfZff7d9YHtq7Ld7U9GjhNi0glShMtfn6ou3COEXa1OlkyvP/file?dl=1
Resolving ucc2519e77d68a8d6f582ee5ace3.dl.dropboxusercontent.com (ucc2519e77d68a8d6f582ee5ace3.dl.dropboxusercontent.com)... 162.125.

In [5]:
import torch

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [6]:
MODEL = 't5-small'
BATCH_SIZE = 48
NUM_PROCS = 16
EPOCHS = 10
OUT_DIR = 'results-t5-medium'
MAX_LENGTH = 256 # Maximum context length to consider while preparing dataset.

In [ ]:
dataset_train = load_dataset(
    'csv',
    data_files='input/train.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='input/valid.csv',
    split='train'
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset_train)
print()
print(dataset_valid)

Dataset({
    features: ['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y'],
    num_rows: 45000
})

Dataset({
    features: ['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y'],
    num_rows: 15000
})


In [ ]:
print(dataset_train[0])

{'Id': 34552656, 'Title': 'Java: Repeat Task Every Random Seconds', 'Body': '<p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print "Hello World" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated.  </p>\n', 'Tags': '<java><repeat>', 'CreationDate': '2016-01-01 00:21:59', 'Y': 'LQ_CLOSE'}


In [7]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"assign tag: {title} {body}" for (title, body) in zip(examples['Title'], examples['Body'])]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    cleaned_tag = [' '.join(''.join(tag.split('<')).split('>')[:-1]) for tag in examples['Tags']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            cleaned_tag,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

Map (num_proc=16):   0%|          | 0/45000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=16):   0%|          | 0/15000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [ ]:
print(tokenized_train[0])

{'Id': 34552656, 'Title': 'Java: Repeat Task Every Random Seconds', 'Body': '<p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print "Hello World" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated.  </p>\n', 'Tags': '<java><repeat>', 'CreationDate': '2016-01-01 00:21:59', 'Y': 'LQ_CLOSE', 'input_ids': [12317, 7860, 10, 10318, 10, 20469, 16107, 2181, 25942, 5212, 7, 3, 2, 102, 3155, 196, 31, 51, 641, 3324, 28, 6103, 53, 4145, 334, 3, 29, 3978, 57, 338, 10318, 5, 13780, 5, 13368, 52, 11, 10318, 5, 13780, 5, 13368, 52, 382, 9, 7, 157, 5, 299, 8857, 497, 27, 241, 12, 2281, 96, 566, 7126, 1150, 121, 12, 8, 8990, 334, 6504, 3978, 45, 209, 4525, 5, 4877, 27, 31, 51, 16, 3, 9, 720, 13, 3, 9, 10505, 11, 278, 31, 17, 43, 136, 1081, 12, 504, 78, 623, 5, 2372, 199, 133, 36, 3, 9, 2246, 4915, 1054, 5, 3, 2, 87, 1

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

60,506,624 total parameters.
60,506,624 training parameters.


In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    save_total_limit=5,
    report_to='tensorboard',
    learning_rate=0.0001,
    fp16=True,
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
history = trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Step,Training Loss,Validation Loss
500,0.090300,0.081005
1000,0.073100,0.066908
1500,0.067000,0.061058
2000,0.062800,0.057801
2500,0.058000,0.055456
3000,0.060900,0.053606
3500,0.052000,0.052144
4000,0.052300,0.050960
4500,0.050800,0.049834
5000,0.051800,0.049123


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

In [ ]:
tokenizer.save_pretrained(OUT_DIR)

('results-t5-medium/tokenizer_config.json',
 'results-t5-medium/special_tokens_map.json',
 'results-t5-medium/spiece.model',
 'results-t5-medium/added_tokens.json')

In [ ]:
!wget "https://www.dropbox.com/scl/fi/9brsjizymq5zvqi7hff09/inference_data.zip?rlkey=ukmdy5egmdld80r5hhmsja78v&dl=1" -O inference_data.zip
!unzip inference_data.zip

--2024-03-28 08:35:08--  https://www.dropbox.com/scl/fi/9brsjizymq5zvqi7hff09/inference_data.zip?rlkey=ukmdy5egmdld80r5hhmsja78v&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9423342ffb8ade0622cedc9921.dl.dropboxusercontent.com/cd/0/inline/CP5FJ_V69Nb7sUifwC_P7Azxz_SuSk7t0VLrSVLUDTDGpsm55lOu8MKaLcm4p4ZD97Y52suhKpu4bUP9DZusZbjzeqP0bpbKSiKostUpnqNKRUBzGiJ7Rmmwa_1GG-ohJEk8ZA0zOEDm1LJrISlJN27T/file?dl=1# [following]
--2024-03-28 08:35:09--  https://uc9423342ffb8ade0622cedc9921.dl.dropboxusercontent.com/cd/0/inline/CP5FJ_V69Nb7sUifwC_P7Azxz_SuSk7t0VLrSVLUDTDGpsm55lOu8MKaLcm4p4ZD97Y52suhKpu4bUP9DZusZbjzeqP0bpbKSiKostUpnqNKRUBzGiJ7Rmmwa_1GG-ohJEk8ZA0zOEDm1LJrISlJN27T/file?dl=1
Resolving uc9423342ffb8ade0622cedc9921.dl.dropboxusercontent.com (uc9423342ffb8ade0622cedc9921.dl.dropboxusercontent.com)...

In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

import os

In [9]:
model_path = 'results-t5-medium/checkpoint-6000/'  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('results-t5-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def tag_generation(text, model, tokenizer):
    input_text = f"assign tag: {text}"
    inputs = tokenizer.encode(
        input_text,
        return_tensors='pt',
        max_length=256,
        padding='max_length',
        truncation=True
    )

    # Get correct sentence ids.
    corrected_ids = model.generate(
        inputs,
        max_length=256,
        num_beams=5, # `num_beams=1` indicated temperature sampling.
        early_stopping=True
    )

    # Decode.
    tags = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    return tags

In [ ]:
for file in os.listdir('inference_data/'):
    f = open(f"inference_data/{file}", 'r')
    sentence = f.read()
    generated_tag = tag_generation(sentence, model, tokenizer)
    print(f"QUERY: {sentence}\nTAGS: {generated_tag}")
    print('-'*100)

QUERY: Repeat Task Every Random Seconds <p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print \"Hello World\" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated

TAGS: java
----------------------------------------------------------------------------------------------------
QUERY: I have a C++ program that I compile on Mac OS 13.4.1 using Cmake

One of my users has the following error: dyld: cannot load 'my_program' (load command 0x80000034 is unknown)

I have no idea why he has this error message, he is on Mac OS 10.14.6 and we both use an Intel Mac

Here are some information about the binary that might be useful:

otool -L my_program
my_program:
        /System/Library/Frameworks/OpenCL.framework/Versions/A/OpenCL (compatibility version 1.0.0, current version 1.0.0)
        /usr/lib/libc++.1.dylib 

In [12]:
# importing required modules
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader('/content/Adenosine.pdf')

# printing number of pages in pdf file
print(len(reader.pages))

content = ""

for page in reader.pages:
    text = page.extract_text()
    content = content + text

print(content)


12
adenocor -ccdsv9 -piv6 -23aug22 Page 1 of 12 AUSTRALIAN PRODUCT I NFORMATION – ADENOCOR® 
(ADENOSINE ) 
1 NAME OF THE MEDICINE  
Adenosine  
2 QUALITATIVE AND QUANTITATIVE COMPOSITION 
Each vial contains 6  mg of adenosine in 2 mL of a 0.9% w/v solution of sodium chloride in 
sterile water for injections.  
For the full list of excipients, see Section 6.1 List of excipients.  
3 PHARMACEUTICAL FORM 
Adenocor is a  clear colourless  sterile solution for intravenous injection (rapid bolus). 
4 CLINICAL PARTICULARS  
4.1 THERAPEUTIC INDICATIONS  
Rapid conversion to a normal sinus rhythm of paroxysmal supraventricular tachycardias, 
including those associated with accessory bypass tracts  (Wolff- Parkinson -White syndrome ). 
Diagnostic Indications  
Aid to diagnosis of broad or narrow QRS complex supraventricular tachycardias.  Although Adenocor  is not effective in converting atrial flutter, atrial fibrillation or ventricular tachycardia 
to sinus rhythm, the slowing of AV conduction

In [13]:
import re

def clean(text):
    text = text.lower()
    text = re.sub('\s*\<.*?\>\s', '', text)
    text = re.sub('\s*\(.*?\)\s', '', text)
    text = re.sub('\s*\[.*?\]\s', '', text)
    text = re.sub('\s*\{.*?\}\s', '', text)
    text = re.sub("[-+@#^/|*(){}$~<>=_%:;]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("\<","",text)
    text = re.sub("\>","",text)
    text = re.sub("\(","",text)
    text = re.sub("\)","",text)
    text = re.sub("[0-9]","",text)
    text = ' '.join(text.split())
    return text

In [15]:
sentence = clean(content)
generated_tag = tag_generation(sentence, model, tokenizer)
print(f"QUERY: {sentence}\nTAGS: {generated_tag}")
print('-'*100)

QUERY: adenocor ccdsv piv aug page of australian product i nformation – adenocor® name of the medicine adenosine qualitative and quantitative composition each vial contains mg of adenosine in ml of a . wv solution of sodium chloride in sterile water for injections. for the full list of excipients, see section . list of excipients. pharmaceutical form adenocor is a clear colourless sterile solution for intravenous injection rapid bolus. clinical particulars . therapeutic indications rapid conversion to a normal sinus rhythm of paroxysmal supraventricular tachycardias, including those associated with accessory bypass tracts wolff parkinson white syndrome . diagnostic indications aid to diagnosis of broad or narrow qrs complex supraventricular tachycardias. although adenocor is not effective in converting atrial flutter, atrial fibrillation or ventricular tachycardia to sinus rhythm, the slowing of av conduction helps diagnosis of atrial activity. in this respect adenosine should be used 